In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from tqdm.auto import tqdm
import numpy as np
from copy import deepcopy

import rospy
import actionlib

from tams_pr2_guzheng.paths import RuckigPath
from tams_pr2_guzheng.utils import *
from tams_pr2_guzheng.msg import RunEpisodeAction
from math import tau

In [ ]:
rospy.init_node('jupyter', anonymous=True)

In [ ]:
run_episode = actionlib.SimpleActionClient('run_episode', RunEpisodeAction)
run_episode.wait_for_server()

In [ ]:
if os.path.exists('/tmp/plucks.json'):
    plucks = pd.read_json('/tmp/plucks.json')
else:
    plucks = pd.DataFrame(columns=[])
plucks

In [ ]:
p = RuckigPath.random(note= 'e4', string_position=.17, direction=1.0)
p.dataframe.plot(x='y', y='z')

In [ ]:
result = run_params(run_episode, p, finger= 'ff')
row = row_from_result(result)
row['params'] = result.parameters
row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
plucks = pd.concat((plucks, row_df), ignore_index=True)

In [ ]:
plucks

In [ ]:
original_p = deepcopy(p)

In [ ]:
p = deepcopy(original_p)
p.dataframe.plot(x='y', y='z')

In [ ]:
def add_noise(path_original):
    path = deepcopy(path_original)
    path.keypoint_pos[0] = np.clip(path.keypoint_pos[0] + np.random.normal(0.0, 1e-3), -.015, 0.015)
    #path.keypoint_pos = np.clip(np.array(path.keypoint_pos) + np.random.normal(0.0, 5e-4, 2), -1.0, 1.0).tolist()
    #vel = np.array(path.keypoint_vel) + np.random.normal(0.0, 1e-3, 2)
    #vel[0] = np.clip(vel[0], *sorted(path.direction * np.array([0, 0.1])))
    #vel[1] = np.clip(vel[1], 0.0, 0.008)
    #path.keypoint_vel = vel.tolist()
    if not path.feasible:
        print("Path not feasible, not modifying")
        return path_original
    return path

In [ ]:
plucks['keypoint_pos_y'].hist()
plt.figure()
plucks['loudness'].hist()

In [ ]:
plt.plot(plucks['loudness'])

In [ ]:
x=plucks.iloc[np.abs(plucks['loudness']-target).argmin()].keypoint_pos_y
print(f"pluck closest to target frequency: {x}")

In [ ]:
# clear NaNs
plucks= plucks[~plucks['loudness'].isnull()]

In [ ]:
p.keypoint_vel

In [ ]:
fig = plt.figure()
# rotate p.keypoint_vel by 10 degrees
for n in np.arange(0.0, tau/4, 5e-2):
    theta = n
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    p.keypoint_vel = (rot @ np.array(original_p.keypoint_vel)).tolist()
    p.dataframe.plot(x='y', y='z', ax = fig.gca(), legend= False)

In [ ]:
np.arange(0.0, tau/4, 5e-2)

In [ ]:
for n in tqdm(np.arange(0.0, 0.1, 2e-3)):
    #p = RuckigPath.from_action_parameters(plucks.iloc[np.abs(plucks['loudness']-target).argmin()]['params'])
    #p = add_noise(p)
    p.keypoint_vel[0] = n
    result = run_params(run_episode, p, finger= 'ff')
    row = row_from_result(result)
    if row['loudness'] is not None:
        row['params'] = result.parameters
        row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
        plucks = pd.concat((plucks, row_df), ignore_index=True)
print(f"\nclosest loudness: {plucks.iloc[np.abs(plucks['loudness']-target).argmin()]['loudness']}")

In [ ]:
p = deepcopy(original_p)
p.keypoint_pos = [0.0, -0.01]
p.keypoint_vel = [0.00, 0.08]
p.dataframe.plot(x='y', y='z', legend=False)

In [ ]:
sns.swarmplot(x= plucks['loudness'])

In [ ]:
plucks.to_json('/tmp/plucks.json')

In [ ]:
target = 25
p = RuckigPath.from_action_parameters(plucks.iloc[np.abs(plucks['loudness']-target).argmin()]['params'])